# 定数を定義(水素分子)

In [1]:
# スピン多重度
multiplicity = 1
# 全電荷
charge = 0
# 原子核配置
distance = 0.977
geometry = [["H", [0, 0, 0]], ["H", [0, 0, distance]]]

# ハミルトニアンを定義(by OpenFermion)

In [2]:
!pip install qulacs pyscf openfermion openfermionpyscf

In [3]:
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.hamiltonians import MolecularData
from openfermionpyscf import run_pyscf
from pyscf import fci

In [4]:
# 分子軌道を表現するための基底関数(Slator Type Orbital - 3 gaussian)
basis = "sto-3g"

# 計算結果の保存場所
description = "tmp"
molecule = MolecularData(geometry, basis, multiplicity, charge, description)
molecule = run_pyscf(molecule, run_scf=1, run_fci=1)

n_qubit = molecule.n_qubits
n_electron = molecule.n_electrons
# get_molecular_hamiltonianで第二量子化形式のハミルトニアンを計算
fermionic_hamiltonian = get_fermion_operator(
    molecule.get_molecular_hamiltonian())
# Jordan-Wigner変換によりハミルトニアンをパウリ行列のテンソル積形式に変換
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)

Jordan-Wigner変換ドキュメント<br>
https://quantumai.google/openfermion/tutorials/jordan_wigner_and_bravyi_kitaev_transforms

In [5]:
print(jw_hamiltonian)

(-0.31349601534094224+0j) [] +
(-0.048834726365406485+0j) [X0 X1 Y2 Y3] +
(0.048834726365406485+0j) [X0 Y1 Y2 X3] +
(0.048834726365406485+0j) [Y0 X1 X2 Y3] +
(-0.048834726365406485+0j) [Y0 Y1 X2 X3] +
(0.13978238294522735+0j) [Z0] +
(0.1576263055158343+0j) [Z0 Z1] +
(0.10745382591353006+0j) [Z0 Z2] +
(0.15628855227893657+0j) [Z0 Z3] +
(0.13978238294522735+0j) [Z1] +
(0.15628855227893657+0j) [Z1 Z2] +
(0.10745382591353006+0j) [Z1 Z3] +
(-0.13686895093682833+0j) [Z2] +
(0.16419290100986608+0j) [Z2 Z3] +
(-0.13686895093682833+0j) [Z3]


# ハミルトニアンを定義(by Qiskit Chemistry)

In [6]:
from qiskit.chemistry.drivers import PySCFDriver, UnitsType, Molecule
from qiskit.chemistry.transformations import FermionicTransformation,FermionicTransformationType, FermionicQubitMappingType

basis = "sto3g"

molecule = Molecule(geometry=geometry, charge=charge, multiplicity=multiplicity)
driver = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM, basis=basis)
transformation = FermionicTransformation(
    transformation=FermionicTransformationType.FULL,
    qubit_mapping=FermionicQubitMappingType.JORDAN_WIGNER,
    two_qubit_reduction=False,
    freeze_core=False)
qubit_op, _ = transformation.transform(driver)
print(qubit_op)

SummedOp([
  -0.8551308269274315 * IIII,
  0.13978238294522738 * IIIZ,
  -0.13686895093682827 * IIZI,
  0.13978238294522735 * IZII,
  -0.13686895093682835 * ZIII,
  0.10745382591353006 * IIZZ,
  0.15762630551583431 * IZIZ,
  0.04883472636540648 * XXYY,
  0.04883472636540648 * YYYY,
  0.04883472636540648 * XXXX,
  0.04883472636540648 * YYXX,
  0.1562885522789365 * ZIIZ,
  0.1562885522789365 * IZZI,
  0.16419290100986614 * ZIZI,
  0.10745382591353006 * ZZII
])
